In [1]:
using Flight
using Unitful
using StaticArrays


Let fh(Oh,εh) denote the construction reference frame used by JSBSim. The origin $O_h$ is an arbitrary point, and the axes $\varepsilon_h$ have x pointing backward and z upward.

Let fb(Ob,εb) denote our main body frame, where εb denote the conventional flight physics axes, x pointing forward and z downward. The rotation from $\varepsilon_b$ to $\varepsilon_h$ is thus given by:

In [2]:
q_bh = REuler(ψ = π, φ = π)
q_hb = q_bh'

REuler(3.141592653589793, -1.4997597826618576e-32, 3.141592653589793)

Now, as our vehicle reference frame origin $O_b$, we select the aerodynamics reference point $O_a$.
Its position is given by JSBSim as:

In [3]:
r_OhOb_h = r_OhOa_h = ustrip.(u"m", [43.2, 0, 59.4]u"inch")

3-element Vector{Float64}:
 1.09728
 0.0
 1.5087599999999999

We can now define the `FrameTransform` between both frames as:

In [4]:
t_hb = FrameTransform(r = r_OhOb_h, q = q_hb)
t_bh = t_hb'

FrameTransform([1.09728, -3.191478544449968e-16, 1.5087599999999999], RQuat([-3.749399456654644e-33, -6.123233995736766e-17, -1.0, -6.123233995736766e-17]))

And a helper function to process raw JSBSim position vectors by first converting them to SI units
and then transforming them to our main reference frame f_b:

In [5]:
fromJSBSim(v::AbstractVector{<:Real}) = ustrip.(u"m", v * u"inch") |> t_bh |> SVector{3,Float64}

fromJSBSim (generic function with 1 method)

With this, we can convert all the following position vectors defined in JSBSim's construction frame to our flight physics conventional reference frame:
- G0 (center of mass for the empty aircraft)
- G1 (center of mass of the pilot)
- G2 (center of mass of copilot)
- G3 (center of mass of left passenger)
- G4 (center of mass of right passenger)
- G5 (center of mass of baggage)
- G6 (center of mass of left fuel tank)
- G7 (center of mass of right fuel tank)
- N (nose landing gear origin)
- L (left main landing gear origin)
- R (right main landing gear origin)
- P (propeller origin)

In [6]:
@show r_ObG0_b = fromJSBSim([41, 0, 36.5])
@show r_ObG1_b = fromJSBSim([36, -14, 24])
@show r_ObG2_b = fromJSBSim([36, 14, 24])
@show r_ObG3_b = fromJSBSim([70, -14, 24])
@show r_ObG4_b = fromJSBSim([70, 14, 24])
@show r_ObG5_b = fromJSBSim([95, 0, 24])
@show r_ObG6_b = fromJSBSim([56, -112, 59.4])
@show r_ObG7_b = fromJSBSim([56, 112, 59.4])
@show r_ObN_b = fromJSBSim([-6.8, 0, -19.5])
@show r_ObL_b = fromJSBSim([58.2, -43, -15.5])
@show r_ObR_b = fromJSBSim([58.2, 43, -15.5])
@show r_ObP_b = fromJSBSim([-37.7, 0, 26.6])

r_ObG0_b = fromJSBSim([41, 0, 36.5]) = [0.05588000000000015, -7.807613203284035e-17, 0.58166]
r_ObG1_b = fromJSBSim([36, -14, 24]) = [0.18288000000000004, -0.3556000000000002, 0.8991599999999998]
r_ObG2_b = fromJSBSim([36, 14, 24]) = [0.18288000000000004, 0.35559999999999986, 0.8991599999999998]
r_ObG3_b = fromJSBSim([70, -14, 24]) = [-0.68072, -0.3556000000000001, 0.8991599999999998]
r_ObG4_b = fromJSBSim([70, 14, 24]) = [-0.68072, 0.35559999999999997, 0.8991599999999998]
r_ObG5_b = fromJSBSim([95, 0, 24]) = [-1.3157199999999998, 5.101388706528218e-17, 0.8991599999999998]
r_ObG6_b = fromJSBSim([56, -112, 59.4]) = [-0.3251200000000003, -2.8447999999999998, -4.440892098500626e-16]
r_ObG7_b = fromJSBSim([56, 112, 59.4]) = [-0.3251199999999994, 2.8447999999999998, 4.440892098500626e-16]
r_ObN_b = fromJSBSim([-6.8, 0, -19.5]) = [1.27, -4.009567099216383e-16, 2.00406]
r_ObL_b = fromJSBSim([58.2, -43, -15.5]) = [-0.381, -1.0922, 1.9024599999999998]
r_ObR_b = fromJSBSim([58.2, 43, -15.5]) = [

3-element SVector{3, Float64} with indices SOneTo(3):
  2.05486
 -3.536755463001573e-16
  0.8331199999999999

From Wikipedia, we know the mass of the empty 172R is 767 kg. The inertia tensor with respect to its own center of mass $G_0$, projected in axes $\varepsilon_b$ (that is, $J_{0[G_0]}^{b}$) can be estimated from the values given by Roskam for a similar configuration but with a different mass (2650 lbs). We can simply scale the inertia values to our actual empty mass. This is not totally accurate, because:
- The airframe dimensions may also be different, not just the mass.
- Some of the mass difference may be due to fuel contents, and these are relatively far from the CG.

In [10]:
# J0_G0_b =  ustrip.(u"kg*m^2", [948 0 0; 0 1346 0; 0 0 1967]u"slug*ft^2")
m0 = 767
J0_G0_b = 894/ustrip(u"kg", 2650u"lb") * ustrip.(u"kg*m^2", [948 0 0; 0 1346 0; 0 0 1967]u"slug*ft^2")
c0 = RigidBodyDistribution(m0, J0_G0_b)

RigidBody(767.0, [955.9509749068076 0.0 0.0; 0.0 1357.2890424309735 0.0; 0.0 0.0 1983.4974342211926])